

## Código Inicial


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
path= "/content/resumen_completo.csv"

datos = pd.read_csv(path, encoding="latin-1")

In [ ]:
lineas= ["LINEA001","LINEA004","LINEA005","LINEA006","LINEA007","LINEA009","MULTI001","JUNTAS"]
compresores= ["COMP01","COMP02","COMP05","COMP07","COMP08","COMP09","COMP10","COMP11"]

## Función Random Forest (Por Linea)

In [ ]:
def randomForestMAPE(nlinea, ncompresor, semilla):
  linea= lineas[nlinea]
  compresor= compresores[ncompresor]

  varCompresor= "MAAP_AUX_REFRIG_" + compresor + "_"
  datos_lineas = datos.loc[datos.LINEA == linea]
  datos_lineas = datos_lineas.loc[datos_lineas[varCompresor+"PE"]!= 0]

  y= datos_lineas[varCompresor+"PE"]
  variables=[varCompresor+"HM",varCompresor+"PA",varCompresor+"PD",varCompresor+"PS",varCompresor+"TD",varCompresor+"TS"]
  X= datos_lineas[variables]
  train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
  model = RandomForestRegressor(random_state=semilla)
  model.fit(train_X, train_y)
  val_predictions = model.predict(val_X)
  actual, pred = np.array(val_y), np.array(val_predictions)
  mape= np.mean(np.abs((actual - pred) / actual)) * 100
  return(mape)

## Función Random Forest (Lineas Juntas)

In [ ]:
def randomForestMAPEJ(ncompresor, semilla):
  compresor= compresores[ncompresor]

  varCompresor= "MAAP_AUX_REFRIG_" + compresor + "_"
  datos_lineas = datos.loc[datos[varCompresor+"PE"]!= 0]

  y= datos_lineas[varCompresor+"PE"]
  variables=[varCompresor+"HM",varCompresor+"PA",varCompresor+"PD",varCompresor+"PS",varCompresor+"TD",varCompresor+"TS"]
  X= datos_lineas[variables]
  train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
  model = RandomForestRegressor(random_state=semilla)
  model.fit(train_X, train_y)
  val_predictions = model.predict(val_X)
  actual, pred = np.array(val_y), np.array(val_predictions)
  mape= np.mean(np.abs((actual - pred) / actual)) * 100
  return(mape)

## Función Gráfico Random Forest (Por Linea)

In [ ]:
def grafRandomForest(nlinea, ncompresor, semilla):
  linea= lineas[nlinea]
  compresor= compresores[ncompresor]

  varCompresor= "MAAP_AUX_REFRIG_" + compresor + "_"
  datos_lineas = datos.loc[datos.LINEA == linea]
  datos_lineas = datos_lineas.loc[datos_lineas[varCompresor+"PE"]!= 0]

  y= datos_lineas[varCompresor+"PE"]
  variables=[varCompresor+"HM",varCompresor+"PA",varCompresor+"PD",varCompresor+"PS",varCompresor+"TD",varCompresor+"TS"]
  X= datos_lineas[variables]
  train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
  model = RandomForestRegressor(random_state=semilla)
  model.fit(train_X, train_y)
  columnas= train_X.columns
  importancia= model.feature_importances_
  tabla_importancia= pd.Series(importancia,columnas)
  return(tabla_importancia)

## Loop de Porcentaje de error absoluto promedio por cada linea en cada compresor

In [ ]:
semilla=100
df_MAPE=np.zeros(shape=(8,8))
for i in range(7):
  for j in range(8):
    df_MAPE[i,j]=randomForestMAPE(i,j,semilla)
for k in range(8):
  df_MAPE[7,k]=randomForestMAPEJ(k,semilla)
df_MAPE= pd.DataFrame(df_MAPE,columns=compresores,index=lineas)
print("Porcentaje de error absoluto promedio para la semilla " + str(semilla))
df_MAPE

Porcentaje de error absoluto promedio para la semilla 100


,COMP01,COMP02,COMP05,COMP07,COMP08,COMP09,COMP10,COMP11
LINEA001,3.843437,27.436808,146.996636,29.957039,189.016878,4.472796,50.467753,16.775918
LINEA004,2.553214,21.087838,74.411128,27.110338,342.290743,22.467203,44.908405,19.431708
LINEA005,2.868967,17.955654,31.835187,38.088698,148.417604,4.678571,22.051449,48.313949
LINEA006,0.782131,79.939223,173.749594,36.436675,104.425787,27.386583,41.857207,7.544877
LINEA007,1.827036,54.225787,50.843061,41.289737,172.366129,8.673816,57.315550,187.535303
LINEA009,1.478911,13.989789,31.643856,60.946279,107.102660,10.647245,80.674056,3.541291
MULTI001,3.026704,25.080738,39.911213,77.459459,144.923856,171.779217,142.964784,89.821794
JUNTAS,0.583513,11.391338,54.655427,35.543585,87.058409,12.482571,113.373824,17.015543
